### EXPORT CSV TO RDD

In [1]:
from pyspark.sql import SparkSession
from pyspark.ml.linalg import Vectors
from pyspark.sql.functions import col

spark = SparkSession.builder.appName("CSVToLibSVM").getOrCreate()


df = spark.read.csv("Datas/events_shot.csv", header=True, inferSchema=True)

your 131072x1 screen size is bogus. expect trouble
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/27 03:26:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
features = ['other_pp','from_fk','from_ti','from_corner','from_counter','from_gk','from_keeper','from_ko',
            'header','corner_type','fk_type','pk_type',
            'half_volley_technique','volley_technique','lob_technique','overhead_technique','backheel_technique','diving_h_technique',
            'distance_to_goal', 'shot_angle', 'preferred_foot_shot', 'under_pressure',
            'shot_aerial_won','shot_first_time','shot_one_on_one','shot_open_goal','shot_follows_dribble','players_inside_area']
target = ['goal']

In [4]:
# Convert each row into LibSVM format
def to_libsvm(row):
    label = row['goal']  # replace with your target column name
    feature_values = [f"{i + 1}:{row[feature]}" for i, feature in enumerate(features)]
    return f"{label} " + " ".join(feature_values)
# Apply the transformation to each row
rdd = df.rdd.map(to_libsvm)
# Save the RDD as a text file
rdd.saveAsTextFile("Datas/libsvm_format.txt")
spark.stop()

### Ensembles RDD BASED API

In [ ]:
from pyspark.mllib.tree import GradientBoostedTrees, GradientBoostedTreesModel
from pyspark.mllib.util import MLUtils
from pyspark.mllib.regression import LabeledPoint
from pyspark import SparkContext

sc = SparkContext(appName="EnsemblesRDDbasedAPI")

your 131072x1 screen size is bogus. expect trouble
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/27 03:30:04 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
data = MLUtils.loadLibSVMFile(sc, "Datas/libsvm_format.txt")
data.take(5)

[LabeledPoint(0.0, (28,[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27],[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13.0231,33.69006,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0])),
 LabeledPoint(0.0, (28,[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27],[0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,14.6031,29.76365,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0])),
 LabeledPoint(0.0, (28,[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27],[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.4904,35.318363,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0])),
 LabeledPoint(0.0, (28,[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27],[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,23.8516,17.43129,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0])),
 LabeledPoint(0.0, (28,[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27],

In [3]:
(trainingData, testData) = data.randomSplit([0.7, 0.3])
model = GradientBoostedTrees.trainClassifier(trainingData,
                                                 categoricalFeaturesInfo={}, numIterations=3)
predictions = model.predict(testData.map(lambda x: x.features))
labelsAndPredictions = testData.map(lambda lp: lp.label).zip(predictions)
testErr = labelsAndPredictions.filter(
        lambda lp: lp[0] != lp[1]).count() / float(testData.count())

24/12/27 03:30:36 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS


In [5]:
print('Test Error = ' + str(testErr))
accuracy = 1 - testErr
print('Accuracy = ' + str(accuracy))
print('Learned classification GBT model:')
print(model.toDebugString())

Test Error = 0.09370780448285174
Accuracy = 0.9062921955171482
Learned classification GBT model:
TreeEnsembleModel classifier with 3 trees

  Tree 0:
    If (feature 19 <= 36.027372)
     If (feature 18 <= 19.387999999999998)
      If (feature 27 <= 1.5)
       Predict: -0.6968841857730746
      Else (feature 27 > 1.5)
       Predict: -0.862046313023485
     Else (feature 18 > 19.387999999999998)
      If (feature 19 <= 16.4827815)
       Predict: -0.9552639898830224
      Else (feature 19 > 16.4827815)
       Predict: -0.8974222694658517
    Else (feature 19 > 36.027372)
     If (feature 8 <= 0.5)
      If (feature 27 <= 1.5)
       Predict: 0.022598870056497175
      Else (feature 27 > 1.5)
       Predict: -0.6033210332103321
     Else (feature 8 > 0.5)
      If (feature 18 <= 5.86815)
       Predict: -0.32098765432098764
      Else (feature 18 > 5.86815)
       Predict: -0.7677979076294973
  Tree 1:
    If (feature 25 <= 0.5)
     If (feature 11 <= 0.5)
      If (feature 19 <= 20.33

In [11]:
from pyspark.mllib.tree import RandomForest, RandomForestModel

model = RandomForest.trainClassifier(trainingData, numClasses=2, categoricalFeaturesInfo={},
                                     numTrees=3, featureSubsetStrategy="auto",
                                     impurity='gini', maxDepth=4, maxBins=32)

# Evaluate model on test instances and compute test error
predictions = model.predict(testData.map(lambda x: x.features))
labelsAndPredictions = testData.map(lambda lp: lp.label).zip(predictions)
testErr = labelsAndPredictions.filter(
    lambda lp: lp[0] != lp[1]).count() / float(testData.count())
print('Test Error = ' + str(testErr))
accuracy = 1 - testErr
print('Accuracy = ' + str(accuracy))
print('Learned classification forest model:')
print(model.toDebugString())

Test Error = 0.09435592762624899
Accuracy = 0.905644072373751
Learned classification forest model:
TreeEnsembleModel classifier with 3 trees

  Tree 0:
    If (feature 18 <= 14.2925)
     If (feature 0 <= 0.5)
      If (feature 27 <= 0.5)
       If (feature 25 <= 0.5)
        Predict: 0.0
       Else (feature 25 > 0.5)
        Predict: 1.0
      Else (feature 27 > 0.5)
       Predict: 0.0
     Else (feature 0 > 0.5)
      If (feature 24 <= 0.5)
       If (feature 21 <= 0.5)
        Predict: 1.0
       Else (feature 21 > 0.5)
        Predict: 0.0
      Else (feature 24 > 0.5)
       Predict: 0.0
    Else (feature 18 > 14.2925)
     If (feature 26 <= 0.5)
      Predict: 0.0
     Else (feature 26 > 0.5)
      If (feature 6 <= 0.5)
       Predict: 0.0
      Else (feature 6 > 0.5)
       Predict: 1.0
  Tree 1:
    If (feature 19 <= 35.972570000000005)
     If (feature 11 <= 0.5)
      If (feature 27 <= 1.5)
       If (feature 25 <= 0.5)
        Predict: 0.0
       Else (feature 25 > 0.5)
  